In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cv2
import sys

In [2]:
## Annotation set
anno_set = 'Set 3/'

## Names of files where mapping is being performed
file1 = '20230313_SE_Lek1_P1D2_DJI_0020'
file2 = '20230313_SE_Lek1_P2D4_DJI_0940'

## Names of files compressed
f1 = 'P1D2_DJI_0020'
f2 = 'P2D4_DJI_0940'

## Column names used often
start_frame1 = 'start_frame_' + f1
start_frame2 = 'start_frame_' + f2
end_frame1 = 'end_frame_' + f1
end_frame2 = 'end_frame_' + f2
lagged_start_frame1 = 'lagged_start_frame_' + f1
lagged_start_frame2 = 'lagged_start_frame_' + f2
lagged_end_frame1 = 'lagged_end_frame_' + f1
lagged_end_frame2 = 'lagged_end_frame_' + f2
Frame1 = 'Frame_' + f1
Frame2 = 'Frame_' + f2
ID1 = 'ID_' + f1
ID2 = 'ID_' + f2

## Boolean to decide if initial frame labelled videos are to be created
preliminary_videos = False

In [3]:
## Read dataframes for both files
df1 = pd.read_csv('/Users/vivekhsridhar/Library/Mobile Documents/com~apple~CloudDocs/Documents/Code/Python/OpenCV/plot_tracks/data/' + file1 + '_tracks.csv')
df1 = df1.loc[:,['frame', 'id', 'bb_left', 'bb_top', 'bb_width', 'bb_height']]
df2 = pd.read_csv('/Users/vivekhsridhar/Library/Mobile Documents/com~apple~CloudDocs/Documents/Code/Python/OpenCV/plot_tracks/data/' + file2 + '_tracks.csv')
df2 = df2.loc[:,['frame', 'id', 'bb_left', 'bb_top', 'bb_width', 'bb_height']]

## Read the manually created mapping file
map_df = pd.read_csv('/Users/vivekhsridhar/Library/Mobile Documents/com~apple~CloudDocs/Documents/Manuscripts/10_dataset/Data/' + anno_set + 'Mapping_' + file1 + '_' + file2 + '.csv', sep=';')
map_df.columns = [Frame1, ID1, Frame2, ID2]
map_df = map_df.sample(frac=1).reset_index(drop=True)

## Frame mapping for overlap (vid2 - vid1) e.g. lag = 3 implies vid1(396) == vid2(399)
lag = 594

In [4]:
## Incorporate lag in the tracking data
if lag > 0:
    df1['lagged_frame'] = df1['frame'] + np.abs(lag)
    df2['lagged_frame'] = df2['frame']
elif lag < 0:
    df1['lagged_frame'] = df1['frame']
    df2['lagged_frame'] = df2['frame'] + np.abs(lag)

In [5]:
## Function to find min and max in an array within the 'current chain'
def find_chain_min_max(array, element):
    ## Find the index of the given element in the array
    index = np.where(array == element)[0][0]
    
    ## Find the starting index of the consecutive chain
    start_index = index
    while start_index > 0 and array[start_index] - array[start_index - 1] == 1:
        start_index -= 1
    
    ## Find the ending index of the consecutive chain
    end_index = index
    while end_index < len(array) - 1 and array[end_index + 1] - array[end_index] == 1:
        end_index += 1
    
    ## Return the minimum and maximum within the consecutive chain
    return array[start_index], array[end_index]

## Function to calculate overlap frames
def calculate_overlap(row):
    start_overlap = max(row[lagged_start_frame1], row[lagged_start_frame2])
    end_overlap = min(row[lagged_end_frame1], row[lagged_end_frame2])
    if start_overlap <= end_overlap:
        return pd.Series([start_overlap, end_overlap])
    else:
        return pd.Series([None, None])

### Write the two videos with frame numbers printed

In [6]:
## Get paths and parameters for creating the output videos
input_vidpath1 = '/Users/vivekhsridhar/Library/Mobile Documents/com~apple~CloudDocs/Documents/Code/Python/OpenCV/plot_tracks/data/' + file1 + '.MP4'
input_vidpath2 = '/Users/vivekhsridhar/Library/Mobile Documents/com~apple~CloudDocs/Documents/Code/Python/OpenCV/plot_tracks/data/' + file2 + '.MP4'
output_vidpath1 = '/Users/vivekhsridhar/Library/Mobile Documents/com~apple~CloudDocs/Documents/Code/Python/OpenCV/plot_tracks/output/tracks/' + anno_set + file1 + '_frameno.mp4'
output_vidpath2 = '/Users/vivekhsridhar/Library/Mobile Documents/com~apple~CloudDocs/Documents/Code/Python/OpenCV/plot_tracks/output/tracks/' + anno_set + file2 + '_frameno.mp4'
codec = 'DIVX'
scaling = 0.5

In [7]:
if preliminary_videos:    
    ## Write output video 1
    ## Open video
    cap = cv2.VideoCapture(input_vidpath1)
    if cap.isOpened() == False:
        sys.exit('Video file cannot be read! Please check input_vidpath to ensure it is correctly pointing to the video file')

    ## Video writer class to output video with contour and centroid of tracked object(s)
    # make sure the frame size matches size of array 'final'
    fourcc = cv2.VideoWriter_fourcc(*codec)
    output_framesize = (int(cap.read()[1].shape[1]*scaling),int(cap.read()[1].shape[0]*scaling))
    out = cv2.VideoWriter(filename = output_vidpath1, fourcc = fourcc, fps = 30.0, frameSize = output_framesize, isColor = True)

    last = 0

    while True:
        ## Capture frame-by-frame
        ret, frame = cap.read()

        this = cap.get(1)

        if ret == True:
            ## Preprocess the image for background subtraction
            frame = cv2.resize(frame, None, fx=scaling, fy=scaling, interpolation=cv2.INTER_LINEAR)

            cv2.putText(frame, str(this), (20,50), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 2)

            ## Display the resulting frame
            out.write(frame)
            cv2.imshow('frame1', frame)
            if cv2.waitKey(1) == 27:
                break

        if last >= this:
            break

        last = this

    ## When everything done, release the capture
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)

In [8]:
if preliminary_videos:
    ## Write output video 2
    ## Open video
    cap = cv2.VideoCapture(input_vidpath2)
    if cap.isOpened() == False:
        sys.exit('Video file cannot be read! Please check input_vidpath to ensure it is correctly pointing to the video file')

    ## Video writer class to output video with contour and centroid of tracked object(s)
    # make sure the frame size matches size of array 'final'
    fourcc = cv2.VideoWriter_fourcc(*codec)
    output_framesize = (int(cap.read()[1].shape[1]*scaling),int(cap.read()[1].shape[0]*scaling))
    out = cv2.VideoWriter(filename = output_vidpath2, fourcc = fourcc, fps = 30.0, frameSize = output_framesize, isColor = True)

    last = 0

    while True:
        ## Capture frame-by-frame
        ret, frame = cap.read()

        this = cap.get(1)

        if ret == True:
            ## Preprocess the image for background subtraction
            frame = cv2.resize(frame, None, fx=scaling, fy=scaling, interpolation=cv2.INTER_LINEAR)

            cv2.putText(frame, str(this), (20,50), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 2)

            ## Display the resulting frame
            out.write(frame)
            cv2.imshow('frame2', frame)
            if cv2.waitKey(1) == 27:
                break

        if last >= this:
            break

        last = this

    ## When everything done, release the capture
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)

### Make a dataframe that has the same rows as  'map_df' but includes start and end frames of each ID in that column

In [9]:
## Ready new mapping dataframe with start and end frames for each ID
for index, row in map_df.iterrows():
    ## Get tracklets from current row
    tracklet1 = df1[df1['id'] == row[ID1]]
    tracklet2 = df2[df2['id'] == row[ID2]]
    
    ## Get start and end frame of tracklets based on annotated frame
    anno_fr1 = row[Frame1]
    anno_fr2 = row[Frame2]
    
    start_fr1, end_fr1 = find_chain_min_max(np.array(tracklet1['frame']), anno_fr1)
    start_fr2, end_fr2 = find_chain_min_max(np.array(tracklet2['frame']), anno_fr2)
    
    end_fr1 = np.max(df1['frame']) if end_fr1 > np.max(df1['frame']) else end_fr1
    end_fr2 = np.max(df2['frame']) if end_fr2 > np.max(df2['frame']) else end_fr2
    
    lagged_start_fr1 = tracklet1.loc[tracklet1['frame'] == start_fr1, 'lagged_frame'].values[0]
    lagged_start_fr2 = tracklet2.loc[tracklet2['frame'] == start_fr2, 'lagged_frame'].values[0]
    lagged_end_fr1 = tracklet1.loc[tracklet1['frame'] == end_fr1, 'lagged_frame'].values[0]
    lagged_end_fr2 = tracklet2.loc[tracklet2['frame'] == end_fr2, 'lagged_frame'].values[0]
    
    ## Compile modified dataframe
    tmp = pd.DataFrame([start_fr1, end_fr1, lagged_start_fr1, lagged_end_fr1, row[ID1], 
                        start_fr2, end_fr2, lagged_start_fr2, lagged_end_fr2, row[ID2]]).T
    tmp.columns = [start_frame1, end_frame1, lagged_start_frame1, lagged_end_frame1, ID1, 
                   start_frame2, end_frame2, lagged_start_frame2, lagged_end_frame2, ID2]
    if index != 0:
        df = pd.concat((df, tmp), axis=0)
    else:
        df = tmp
        
df = df.reset_index(drop = True)
df = df.drop_duplicates()

Errors detected were manually deleted in the mapping file. These rows were deleted because the frame indicated in the map_df dataframe was outside of the life of the corresponding ID.

I also manually added lines at the end of the mapping files.

In [10]:
## Calculate overlap between the two videos
overlap_frames = df.apply(calculate_overlap, axis=1)
overlap_frames.columns = ['start_frame_overlap', 'end_frame_overlap']

## Add the calculated overlap frames to the original DataFrame
df = pd.concat([df, overlap_frames], axis=1)
df['overlap_duration'] = df['end_frame_overlap'] - df['start_frame_overlap'] + 1

### Plot videos of the two videos with colours to indicate corresponding tracklets in the two videos

In [11]:
## Get paths and parameters for creating the output videos
output_vidpath1 = '/Users/vivekhsridhar/Library/Mobile Documents/com~apple~CloudDocs/Documents/Code/Python/OpenCV/plot_tracks/output/tracks/' + anno_set + file1 + '_tracked.mp4'
output_vidpath2 = '/Users/vivekhsridhar/Library/Mobile Documents/com~apple~CloudDocs/Documents/Code/Python/OpenCV/plot_tracks/output/tracks/' + anno_set + file2 + '_tracked.mp4'
output_vidpath_merged = '/Users/vivekhsridhar/Library/Mobile Documents/com~apple~CloudDocs/Documents/Code/Python/OpenCV/plot_tracks/output/tracks/' + anno_set + file1 + '_' + file2 + '_merged.mp4'

## Plotting parameters
max_radius = 5
col_mult = int(255/len(df))
colours = [(i*col_mult,255,255) for i in range(len(df))]

## Include the colour column to df
df['colour'] = colours

In [12]:
## Calculate positions of all individuals in both videos
df1['pos_x'] = (df1['bb_left'] + df1['bb_width']/2)*scaling
df1['pos_y'] = (df1['bb_top'] + df1['bb_height']/2)*scaling
df2['pos_x'] = (df2['bb_left'] + df2['bb_width']/2)*scaling
df2['pos_y'] = (df2['bb_top'] + df2['bb_height']/2)*scaling

## Number of frames to plot
fr_min = int(np.min([np.min(df1['lagged_frame']), np.min(df2['lagged_frame'])]))
fr_max = int(np.max([np.max(df1['lagged_frame']), np.max(df2['lagged_frame'])]))
N = fr_max - fr_min + 1

In [13]:
## Write video 1
## Read input video
cap = cv2.VideoCapture(input_vidpath1)
if cap.isOpened() == False:
    sys.exit('Video file cannot be read! Please check input_vidpath to ensure it is correctly pointing to the video file')

## Get properties of the input video
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)*scaling)
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)*scaling)

## Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*codec)  # Specify codec
out = cv2.VideoWriter(output_vidpath1, fourcc, fps, (width, height))

## Prepare a black frame
black_frame = np.zeros((height, width, 3), np.uint8)

## Create a dictionary for quick lookup of frames to be copied
if lag > 0:
    lagged_frames = set(df1['lagged_frame'].tolist())
elif lag < 0:
    lagged_frames = set(df2['lagged_frame'].tolist())

## Initialize a counter for reading frames from input video
input_frame_idx = 0

for i in range(N):
    if i in lagged_frames:
        ret, frame = cap.read()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        if ret:
            frame = cv2.resize(frame, None, fx=scaling, fy=scaling, interpolation=cv2.INTER_LINEAR)
            
            ## Filter objects of interest based on the current frame
            filtered_df = df[(df['start_frame_overlap'] <= i) & (df['end_frame_overlap'] >= i)]
            if filtered_df.empty == False:
                indices, interested_objects = zip(*[(idx, row[ID1]) for idx, row in filtered_df.iterrows()])

                ## Filter MOT dataframe based on the current frame and interested objects
                tmp = df1[(df1['lagged_frame'] == i) & (df1['id'].isin(interested_objects))]
                tmp = tmp.reset_index()

                for j in range(0,len(tmp)):
                    x = int(tmp.loc[j,'pos_x'])
                    y = int(tmp.loc[j,'pos_y'])
                    r = 7
                    c = df.loc[indices[np.where(interested_objects == tmp.loc[j,'id'])[0][0]], 'colour']
                    cv2.circle(frame, (x,y), r, c, -1, cv2.LINE_AA)
                    cv2.putText(frame, str(int(tmp.loc[j,'id'])), (x+20,y-20), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2)
            
            cv2.putText(frame, str(input_frame_idx), (20,50), cv2.FONT_HERSHEY_PLAIN, 3, (0, 0, 255), 2)
            frame = cv2.cvtColor(frame, cv2.COLOR_HSV2BGR)
            
            out.write(frame)
            cv2.imshow('output1', frame)  ## Display the frame
            input_frame_idx += 1
        else:
            ## In case there's an issue reading a frame, write a black frame
            out.write(black_frame)
            cv2.imshow('output1', black_frame)  ## Display the black frame
    else:
        out.write(black_frame)
        cv2.imshow('output1', black_frame)  ## Display the black frame
        
    ## Exit if the user presses the 'q' key
    if cv2.waitKey(1) == 27:
        break

## Release everything if job is finished
cap.release()
out.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


-1

In [ ]:
## Write video 2
## Read input video
cap = cv2.VideoCapture(input_vidpath2)
if cap.isOpened() == False:
    sys.exit('Video file cannot be read! Please check input_vidpath to ensure it is correctly pointing to the video file')

## Get properties of the input video
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)*scaling)
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)*scaling)

## Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*codec)  # Specify codec
out = cv2.VideoWriter(output_vidpath2, fourcc, fps, (width, height))

## Prepare a black frame
black_frame = np.zeros((height, width, 3), np.uint8)

## Create a dictionary for quick lookup of frames to be copied
if lag > 0:
    lagged_frames = set(df2['lagged_frame'].tolist())
elif lag < 0:
    lagged_frames = set(df1['lagged_frame'].tolist())

## Initialize a counter for reading frames from input video
input_frame_idx = 0

for i in range(N):
    if i in lagged_frames:
        ret, frame = cap.read()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        if ret:
            frame = cv2.resize(frame, None, fx=scaling, fy=scaling, interpolation=cv2.INTER_LINEAR)
            
            ## Filter objects of interest based on the current frame
            filtered_df = df[(df['start_frame_overlap'] <= i) & (df['end_frame_overlap'] >= i)]
            if filtered_df.empty == False:
                indices, interested_objects = zip(*[(idx, row[ID2]) for idx, row in filtered_df.iterrows()])

                ## Filter MOT dataframe based on the current frame and interested objects
                tmp = df2[(df2['lagged_frame'] == i) & (df2['id'].isin(interested_objects))]
                tmp = tmp.reset_index()

                for j in range(0,len(tmp)):
                    x = int(tmp.loc[j,'pos_x'])
                    y = int(tmp.loc[j,'pos_y'])
                    r = 7
                    c = df.loc[indices[np.where(interested_objects == tmp.loc[j,'id'])[0][0]], 'colour']
                    cv2.circle(frame, (x,y), r, c, -1, cv2.LINE_AA)
                    cv2.putText(frame, str(int(tmp.loc[j,'id'])), (x+20,y-20), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2)
            
            cv2.putText(frame, str(input_frame_idx), (20,50), cv2.FONT_HERSHEY_PLAIN, 3, (0, 0, 255), 2)
            frame = cv2.cvtColor(frame, cv2.COLOR_HSV2BGR)
            
            out.write(frame)
            cv2.imshow('output2', frame)  ## Display the frame
            input_frame_idx += 1
        else:
            ## In case there's an issue reading a frame, write a black frame
            out.write(black_frame)
            cv2.imshow('output2', black_frame)  ## Display the black frame
    else:
        out.write(black_frame)
        cv2.imshow('output2', black_frame)  ## Display the black frame
        
    ## Exit if the user presses the 'q' key
    if cv2.waitKey(1) == 27:
        break

## Release everything if job is finished
cap.release()
out.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [ ]:
## Write the concatenated video
## Read both input videos
cap1 = cv2.VideoCapture(output_vidpath1)
if not cap1.isOpened():
    sys.exit('Video file 1 cannot be read! Please check input_vidpath1 to ensure it is correctly pointing to the video file')

cap2 = cv2.VideoCapture(output_vidpath2)
if not cap2.isOpened():
    sys.exit('Video file 2 cannot be read! Please check input_vidpath2 to ensure it is correctly pointing to the video file')

## Video writer class to output concatenated video
fourcc = cv2.VideoWriter_fourcc(*codec)
output_framesize = (int(cap1.get(cv2.CAP_PROP_FRAME_WIDTH) * scaling), int(cap1.get(cv2.CAP_PROP_FRAME_HEIGHT) * scaling * 2))
out = cv2.VideoWriter(filename=output_vidpath_merged, fourcc=fourcc, fps=30.0, frameSize=output_framesize, isColor=True)

## Loop through frames of both videos simultaneously and concatenate vertically
while(True):
    ret1, frame1 = cap1.read()
    ret2, frame2 = cap2.read()

    ## Check if either video has reached the end
    if not ret1 or not ret2:
        break

    ## Resize frames
    frame1 = cv2.resize(frame1, None, fx=scaling, fy=scaling, interpolation=cv2.INTER_LINEAR)
    frame2 = cv2.resize(frame2, None, fx=scaling, fy=scaling, interpolation=cv2.INTER_LINEAR)

    ## Concatenate frames vertically
    concatenated_frame = cv2.vconcat([frame1, frame2])

    ## Write the concatenated frame to the output video
    out.write(concatenated_frame)

    ## Display the concatenated frame
    cv2.imshow('output_merged', concatenated_frame)
    if cv2.waitKey(1) == 27:
        break

## When everything done, release the capture
cap.release()
out.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

In [ ]:
## Save modified dataframe
df = df.drop(['start_frame_overlap', 'end_frame_overlap'], axis=1)
df.to_csv('/Users/vivekhsridhar/Library/Mobile Documents/com~apple~CloudDocs/Documents/Code/Python/OpenCV/plot_tracks/output/tracks/' + anno_set + file1 + '_' + file2 + '.csv', mode='w')